<a href="https://colab.research.google.com/github/rishuatgithub/MLPy/blob/master/WordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [0]:
embedding_layers = layers.Embedding(1000,10)

In [6]:
result = embedding_layers(tf.constant([1,2,3]))
result.numpy()

array([[ 0.04143567,  0.03444126, -0.00433236, -0.0115747 ,  0.00946363,
         0.03984356, -0.01140376, -0.01553496,  0.045456  , -0.0431954 ],
       [-0.02294084,  0.00314428, -0.01986251, -0.02078898,  0.01738504,
        -0.03874723,  0.04903078, -0.00269741,  0.01278008, -0.03104183],
       [-0.0045128 ,  0.0245247 ,  0.02320014,  0.03158582,  0.01212316,
        -0.03494855,  0.0398595 ,  0.0347522 , -0.03337281, -0.01772972]],
      dtype=float32)